# Demo ShiftAmountActivity
This activity uses a processor to transfer a specified number of objects from an origin resource, which must have a container, to a destination resource, which also must have a container.

Shift Amount checks the possible amount of objects which can be transferred, based on the number of objects available in the origin, the number of objects which can be stored in the destination and the number of objects requested to be transferred. If the number of actually to be transferred objects is zero than an exception is raised. These cases have to be prevented by using appropriate events.

In [1]:
import datetime, time
import simpy

import shapely.geometry

import pandas as pd
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

In [2]:
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}

In [3]:
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.HasResource,
        core.LoadingFunction,
        core.UnloadingFunction,
    ),
    {},
)

location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)

## Definition of Sites

In [4]:
from_site = Site(
    env=my_env,
    name="Winlocatie",
    geometry=location_from_site,
    capacity=10,
    level=2,
)

## Creation of Vessel

In [5]:
hopper = TransportProcessingResource(
    env=my_env,
    name="Hopper 01",
    geometry=location_from_site,
    loading_rate=1,
    unloading_rate=1,
    capacity=5,
    compute_v=lambda x: 10,
)

## Definition of Shift Amount activity

In [6]:
shift_amount_activity_data = model.ShiftAmountActivity(
    env=my_env,
    name="Transfer MP",
    ID="6dbbbdf7-4589-11e9-bf3b-b469212bff5b",
    registry=registry,
    processor=hopper,
    origin=from_site,
    destination=hopper,
    amount=100,
    duration=10,
)

## Run simulation

In [7]:
model.register_processes([shift_amount_activity_data])
my_env.run()

In [9]:
display(plot.get_log_dataframe(shift_amount_activity_data, [shift_amount_activity_data]))
display(plot.get_log_dataframe(from_site, [shift_amount_activity_data]))
display(plot.get_log_dataframe(hopper, [shift_amount_activity_data]))

,Activity,Timestamp,ActivityState
0,Transfer MP,1970-01-01 00:00:00,START
1,Transfer MP,1970-01-01 00:00:10,STOP


,Activity,Timestamp,ActivityState,container level,geometry
0,Transfer MP,1970-01-01 00:00:00,START,2,POINT (4.18055556 52.18664444)
1,Transfer MP,1970-01-01 00:00:10,STOP,0,POINT (4.18055556 52.18664444)


,Activity,Timestamp,ActivityState,geometry,container level
0,Transfer MP,1970-01-01 00:00:00,START,POINT (4.18055556 52.18664444),0.0
1,Transfer MP,1970-01-01 00:00:00,START,POINT (4.18055556 52.18664444),0.0
2,Transfer MP,1970-01-01 00:00:10,STOP,POINT (4.18055556 52.18664444),2.0
3,Transfer MP,1970-01-01 00:00:10,STOP,POINT (4.18055556 52.18664444),2.0


The resulting levels ob objects in the hopper and the from_site are requested below. 